In [5]:
import numpy as np
from pyscf import gto, scf
from pyblock2.driver import core
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from CustomHamil import HubbardHamiltonian

N = 4
L = 4
U = 2
omega = 5.0
lamb = 3.6
g = np.sqrt(lamb * omega / 2)

nbasis_fermion = L
nbasis_boson = L
nbcuts = [3] * L

h1 = np.zeros((N, N))
for i in range(N-1):
    h1[i, i+1] = h1[i+1, i] = -1.0
# h1[N-1, 0] = h1[0, N-1] = -1.0  # PBC
eri = np.zeros((N, N, N, N))
for i in range(N):
    eri[i, i, i, i] = U

gmat = np.zeros((N, N, N))
for i in range(N):
    gmat[i, i, i] = g

omat = np.array([omega] * L)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
b = driver.expr_builder()
driver.initialize_system(n_sites=nbasis_fermion+nbasis_boson, n_elec=L, spin=0, orb_sym=None)

print(driver.n_sites)

driver.ghamil = HubbardHamiltonian(driver, driver.vacuum, driver.n_sites, driver.orb_sym, n_sites_fermion=nbasis_fermion, n_sites_boson=nbasis_boson, nbcuts=nbcuts)
mpo = driver.get_qc_eph_mpo(h1, eri, gmat, omat)

ket = driver.get_random_mps(tag="GS", bond_dim=500, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [500] * 4 + [100] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-9] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=100,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)


8
mpo terms =         44

Build MPO | Nsites =     8 | Nterms =         44 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     8 .. Mmpo =     8 DW = 0.00e+00 NNZ =        8 SPT = 0.0000 Tmvc = 0.000 T = 0.004
 Site =     1 /     8 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8125 Tmvc = 0.000 T = 0.013
 Site =     2 /     8 .. Mmpo =    12 DW = 0.00e+00 NNZ =       17 SPT = 0.8583 Tmvc = 0.000 T = 0.008
 Site =     3 /     8 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8750 Tmvc = 0.000 T = 0.009
 Site =     4 /     8 .. Mmpo =     8 DW = 0.00e+00 NNZ =       11 SPT = 0.8625 Tmvc = 0.001 T = 0.012
 Site =     5 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =        9 SPT = 0.8125 Tmvc = 0.000 T = 0.003
 Site =     6 /     8 .. Mmpo =     4 DW = 0.00e+00 NNZ =        7 SPT = 0.7083 Tmvc = 0.000 T = 0.012
 Site =     7 /     8 .. Mmpo =     1 DW = 0.00e+00 NNZ =        4 SPT = 0.0000 Tmvc = 0.000 T = 0.002
Ttotal =      0.063 Tmvc-total = 0.001 MPO bond dimension =

In [6]:
energies

-11.047212489667903